In [1]:
from keras.applications import MobileNetV2

img_rows, img_cols = 224,224

MobileNetV2_model = MobileNetV2(weights='imagenet',
                               include_top=False,
                               input_shape=(img_rows,img_cols,3))

for layer in MobileNetV2_model.layers:
    layer.trainable = False

In [2]:
for i,layer in enumerate(MobileNetV2_model.layers):
    print(str(i)+' '+layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 Conv2D False
11 BatchNormalization False
12 ReLU False
13 ZeroPadding2D False
14 DepthwiseConv2D False
15 BatchNormalization False
16 ReLU False
17 Conv2D False
18 BatchNormalization False
19 Conv2D False
20 BatchNormalization False
21 ReLU False
22 DepthwiseConv2D False
23 BatchNormalization False
24 ReLU False
25 Conv2D False
26 BatchNormalization False
27 Add False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 ZeroPadding2D False
32 DepthwiseConv2D False
33 BatchNormalization False
34 ReLU False
35 Conv2D False
36 BatchNormalization False
37 Conv2D False
38 BatchNormalization False
39 ReLU False
40 DepthwiseConv2D False
41 BatchNormalization False
42 ReLU False
43 Conv2D False
44 BatchNormalization False
45 Add False
46 Conv2D False
47 BatchNormalization False
48 ReL

In [3]:
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model

In [4]:
def addTopModelMobileNet(bottom_model,num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    
    return top_model

In [5]:
num_class = 10

FC_Head = addTopModelMobileNet(MobileNetV2_model,num_class)

model = Model(inputs=MobileNetV2_model.inputs , outputs=FC_Head)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

## step 1. Loading our Monkey Breed Dataset

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [33]:
train_data_dir = '../../../../data_for_computer_vision/datasets/monkey_breed/train/'
test_data_dir = '../../../../data_for_computer_vision/datasets/monkey_breed/validation/'

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=45,
                                  width_shift_range=0.3,
                                  height_shift_range=0.3,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size =32
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical')

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


## step 2. create Checkpoints

In [19]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import RMSprop

checkpoint = ModelCheckpoint('monkey_breed_mobileNet.h5',
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights=True)

callbacks = [earlystop,checkpoint]

## step 3. create and training model

In [20]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=0.001),
             metrics=['accuracy'])

num_of_train_sample = 1098
num_of_validation_sample = 272

# train 1 epoch
epochs = 1
batch_size = 16

history = model.fit_generator(train_generator,
                             steps_per_epoch=num_of_train_sample//batch_size,
                             epochs=epochs,
                             callbacks=callbacks,
                             validation_data=validation_generator,
                             validation_steps=num_of_validation_sample//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
35/68 [==============>...............] - ETA: 26s - loss: 1.8823 - accuracy: 0.5337WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 68 batches). You may need to use the repeat() function when building your dataset.

Epoch 00001: val_loss improved from inf to 0.69784, saving model to monkey_breed_mobileNet.h5
35/68 [==============>...............] - 36s 1s/step - loss: 1.8823 - accuracy: 0.5337 - val_loss: 0.6978 - val_accuracy: 0.7978


## step 4. Test our model.

In [26]:
import os
import numpy as np

In [21]:
monkey_breeds_dict = {"[0]": "mantled_howler ", 
                      "[1]": "patas_monkey",
                      "[2]": "bald_uakari",
                      "[3]": "japanese_macaque",
                      "[4]": "pygmy_marmoset ",
                      "[5]": "white_headed_capuchin",
                      "[6]": "silvery_marmoset",
                      "[7]": "common_squirrel_monkey",
                      "[8]": "black_headed_night_monkey",
                      "[9]": "nilgiri_langur"}

monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                      "n1": "patas_monkey",
                      "n2": "bald_uakari",
                      "n3": "japanese_macaque",
                      "n4": "pygmy_marmoset ",
                      "n5": "white_headed_capuchin",
                      "n6": "silvery_marmoset",
                      "n7": "common_squirrel_monkey",
                      "n8": "black_headed_night_monkey",
                      "n9": "nilgiri_langur"}

In [31]:
def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)
    
def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.listdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)  

In [35]:
for i in range(0,10):
    input_im = getRandomImage(test_data_dir)
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(model.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - silvery_marmoset
Class - japanese_macaque
Class - mantled_howler 
Class - patas_monkey
Class - common_squirrel_monkey
Class - silvery_marmoset
Class - patas_monkey
Class - common_squirrel_monkey
Class - silvery_marmoset
Class - silvery_marmoset
